In [37]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import os

In [38]:
import tensorflow as tf

# Allow GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU available.")

# Initialize GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Physical devices cannot be modified after being initialized
Num GPUs Available:  1


Descarga de datos


In [39]:
path_to_fileDL = tf.keras.utils.get_file('DL-Introduccion-practica-con-Keras-1a.txt', 'https://www.gutenberg.org/cache/epub/345/pg345.txt')

text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:    {} caracteres'.format(len(text)))

vocab = sorted(set(text))
print ('El texto esta compuesto de estos {} caracteres'.format(len(vocab)))
print (vocab)

Longitud del texto:    881631 caracteres
El texto esta compuesto de estos 106 caracteres
['\n', '\r', ' ', '!', '#', '$', '%', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '£', 'à', 'á', 'â', 'æ', 'è', 'é', 'ë', 'ï', 'ô', 'ö', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


Tablas de traduccion o Inversa de vocabulario


In [40]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [41]:
for char,_ in zip(char2idx, range(len(vocab))):
  print(' {:4s}: {:3d},'.format(repr(char), char2idx[char]))

 '\n':   0,
 '\r':   1,
 ' ' :   2,
 '!' :   3,
 '#' :   4,
 '$' :   5,
 '%' :   6,
 '&' :   7,
 '(' :   8,
 ')' :   9,
 '*' :  10,
 ',' :  11,
 '-' :  12,
 '.' :  13,
 '/' :  14,
 '0' :  15,
 '1' :  16,
 '2' :  17,
 '3' :  18,
 '4' :  19,
 '5' :  20,
 '6' :  21,
 '7' :  22,
 '8' :  23,
 '9' :  24,
 ':' :  25,
 ';' :  26,
 '=' :  27,
 '>' :  28,
 '?' :  29,
 'A' :  30,
 'B' :  31,
 'C' :  32,
 'D' :  33,
 'E' :  34,
 'F' :  35,
 'G' :  36,
 'H' :  37,
 'I' :  38,
 'J' :  39,
 'K' :  40,
 'L' :  41,
 'M' :  42,
 'N' :  43,
 'O' :  44,
 'P' :  45,
 'Q' :  46,
 'R' :  47,
 'S' :  48,
 'T' :  49,
 'U' :  50,
 'V' :  51,
 'W' :  52,
 'X' :  53,
 'Y' :  54,
 'Z' :  55,
 '[' :  56,
 ']' :  57,
 '_' :  58,
 'a' :  59,
 'b' :  60,
 'c' :  61,
 'd' :  62,
 'e' :  63,
 'f' :  64,
 'g' :  65,
 'h' :  66,
 'i' :  67,
 'j' :  68,
 'k' :  69,
 'l' :  70,
 'm' :  71,
 'n' :  72,
 'o' :  73,
 'p' :  74,
 'q' :  75,
 'r' :  76,
 's' :  77,
 't' :  78,
 'u' :  79,
 'v' :  80,
 'w' :  81,
 'x' :  82,
 'y'

convertir texto a enteros

In [42]:
text_as_int = np.array([char2idx[c] for c in text])

In [43]:
#Mostramos algunos caracteres
print('text: {}'.format(repr(text[:50])))
print('{}'.format(repr(text_as_int[:50])))

text: '\ufeffThe Project Gutenberg eBook of Dracula\r\n    \r\nThi'
array([105,  49,  66,  63,   2,  45,  76,  73,  68,  63,  61,  78,   2,
        36,  79,  78,  63,  72,  60,  63,  76,  65,   2,  63,  31,  73,
        73,  69,   2,  73,  64,   2,  33,  76,  59,  61,  79,  70,  59,
         1,   0,   2,   2,   2,   2,   1,   0,  49,  66,  67])


PREPARAR DATOS

In [44]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

seq_length = 100

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [45]:
#comprobar datos
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'\ufeffThe Project Gutenberg eBook of Dracula\r\n    \r\nThis ebook is for the use of anyone anywhere in the Un'
'ited States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever'
'. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License includ'
'ed with this ebook or online\r\nat www.gutenberg.org. If you are not located in the United States,\r\nyou'
' will have to check the laws of the country where you are located\r\nbefore using this eBook.\r\n\r\nTitle:'
' Dracula\r\n\r\n\r\nAuthor: Bram Stoker\r\n\r\nRelease date: October 1, 1995 [eBook #345]\r\n                Most'
' recently updated: July 30, 2023\r\n\r\nLanguage: English\r\n\r\n\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK'
' DRACULA ***\r\n\r\n\r\n\r\n                                DRACULA\r\n\r\n                                  _by_'
'\r\n\r\n                              Bram Stoker\r\n\r\n                        [Illustration: colophon]\

In [46]:
#Preparar datos de entrenamiento  (Entrada 0 a 99 )  (Salida 1 a 100)
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [47]:
#Visualizamos
for input_example, target_example in dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\ufeffThe Project Gutenberg eBook of Dracula\r\n    \r\nThis ebook is for the use of anyone anywhere in the U'
Target data:  'The Project Gutenberg eBook of Dracula\r\n    \r\nThis ebook is for the use of anyone anywhere in the Un'


In [48]:
#imprimir dataset
print (dataset)

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int32, name=None), TensorSpec(shape=(100,), dtype=tf.int32, name=None))>


In [49]:
#agrupar en batches
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>


Construir modelo RNN

In [50]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                batch_input_shape=[batch_size,None]),

      tf.keras.layers.LSTM(rnn_units,
                           return_sequences=True,
                           stateful = True,
                           recurrent_initializer='glorot_uniform'),

      tf.keras.layers.Dense(vocab_size)
  ])
  return model

vocab_size = len(vocab)
embedding_dim= 256
rnn_units = 1024

model = build_model(
    vocab_size = vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size = BATCH_SIZE
)

In [51]:
#Visualizar estructura
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 256)           27136     
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_2 (Dense)             (64, None, 106)           108650    
                                                                 
Total params: 5,382,762
Trainable params: 5,382,762
Non-trainable params: 0
_________________________________________________________________


In [52]:
# Forma de input
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input: ", input_example_batch.shape, "# (batch_size, lenght)")
  print("Target: ", target_example_batch.shape, "# (batch_size, sequence_length)")

Input:  (64, 100) # (batch_size, lenght)
Target:  (64, 100) # (batch_size, sequence_length)


In [53]:
#Forma de salida
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


Prediction:  (64, 100, 106) # (batch_size, sequence_length, vocab_size)


In [54]:
#Mostar que el resultado es una distribucion, no un argmax

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()
print(sampled_indices_characters)

[ 39  46  14  58  44   4  36  99  60  99  41  58   5 102  76   7  82  66
  28 102  41  64  25  35  36  81   0  26  22  81   5  67  43   6  87  56
  65  45  31  77   9  80  84  95  64   8  24  94  47  10  35  97  31  53
  62  15  93  50 101  14  30  24  94  91  46 101  30  78  30  59   5  34
 104  74  89  94  79  72 102   0   2  69  21  80  71  67  29  67   7 102
   6  78  71   8  11  84 102  46  48  60]


ENTRENAMIENTO

In [55]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)


In [56]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_(epoch)")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [58]:
EPOCHS = 10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10


136/136 [==============================] - 23s 156ms/step - loss: 2.4116
Epoch 2/10
136/136 [==============================] - 23s 161ms/step - loss: 2.0912
Epoch 3/10
136/136 [==============================] - 23s 156ms/step - loss: 1.8965
Epoch 4/10
136/136 [==============================] - 24s 161ms/step - loss: 1.7555
Epoch 5/10
136/136 [==============================] - 23s 155ms/step - loss: 1.6513
Epoch 6/10
136/136 [==============================] - 23s 161ms/step - loss: 1.5713
Epoch 7/10
136/136 [==============================] - 23s 157ms/step - loss: 1.5095
Epoch 8/10
136/136 [==============================] - 22s 153ms/step - loss: 1.4620
Epoch 9/10
136/136 [==============================] - 24s 161ms/step - loss: 1.4230
Epoch 10/10
136/136 [==============================] - 23s 157ms/step - loss: 1.3889


Generacion de texto

In [59]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1,None]))

In [60]:
def generate_text(model, start_string, temp):
  num_generate = 500
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []

  temperature = temp

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)

    predictions = tf.squeeze(predictions,0)

    predictions = predictions/temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    input_eval = tf.expand_dims([predicted_id],0)

    text_generated.append(idx2char[predicted_id])

  return(start_string + ''.join(text_generated))

In [61]:
print(generate_text(model, start_string=u"Dracula", temp=0.5))

Dracula•}öè$—ö﻿ööâ>öö£öööéööööë‘”ëX﻿öööööè﻿•﻿ööö}ëöööö﻿£•öH}﻿xö﻿[﻿﻿ööööè£ööë£ë﻿ö£﻿ööööö>ööX£>öë#﻿ö£ë#ö﻿ö#‘﻿﻿öëèëëXööëööö﻿öë>ööë‘}#öööâöëö#öö﻿﻿﻿öö﻿öö#è[öööè£èö﻿ö>öâ[ö£﻿öëö﻿éööèè™‘àöë•&ö}£öö>﻿öè—öè﻿öà>ö&‘ö﻿áö—ö£ëëöööëö#ë‘—ï﻿﻿ö£öë£ö﻿﻿X£[﻿öö#ö*™ëëëF}ö0ö£ö>ö$ö﻿﻿öâ£﻿ö#öö﻿öëö}öXëö}öö﻿﻿ë﻿ëöö—ö﻿âöëXöæèö﻿ëXé>öë]öö﻿ë>#öö﻿ö[öö﻿﻿﻿öâööö﻿﻿﻿﻿ëX—}ëöèöööZë>è﻿﻿èëöëö‘âö﻿﻿öáöXö﻿ö﻿ööööö﻿ö﻿﻿﻿è™öëööá>﻿éö﻿”>öëööööö([Xööà﻿﻿ë—ë﻿ö}éöö﻿#ö﻿ö]}ëáöööööè﻿è﻿ö}öëXö—ë﻿ööXö﻿﻿ëöèöö﻿ë﻿ö﻿âö•‘%﻿ëööö™>à‘}é[ö﻿è﻿ööxö﻿öööööö﻿ö﻿﻿ö﻿﻿ëëöö—™>£öö﻿ö﻿é﻿


In [62]:
print(generate_text(model, start_string=u"Dracula", temp=0.8))

Dracula[£ë‘﻿]ö‘™™èëöööö£âö>£﻿ö﻿öö£ëöj£﻿ö(>﻿éëööéö™ö—ö%[ö﻿ëX﻿ö”>£ëï#£èX[$﻿﻿ö£â[èöë#﻿£âàëjö%‘%£â&‘öâë‘Z($[0ë‘0z﻿ö}>>ö(—xïö﻿ö0ö•ë‘èZè#àZöë}ö0è﻿#﻿ôè﻿>ë*ö•[é}éö[öëöö﻿ööëX[ö﻿ö﻿ô£ö•é‘àëDöæ£ëïöëZ﻿ë9#ö£ö﻿èöà&>﻿ë}ëïZ•ëöëâë‘öéö£èëâëà﻿è”﻿x>ëëö﻿﻿ë}#﻿}£•èö﻿ë#ö£èö£é﻿öö£*xöé>>﻿ëö[Zëöj}ëZööà>>£ö—é}ô﻿ëöX•£ëö0>ö•öï﻿é﻿%*$ö#Zë£ö﻿ëö—{™éö}ë>>>&öâö[﻿öö£ëï#ë#)ë—èàXXö﻿9ë﻿ö£Zöàëö﻿7â™W£xö9öëèöZ&öà&‘àö﻿ë*>#﻿0*>è[0â>öéè#﻿}ë£ëj﻿âè}é>ï•ë}&ë﻿xö##>ö%#}ë[è™ëö£>ö[ööö”﻿﻿﻿è[öXöëö﻿âZö%ö﻿%‘(>£•*âööëöéö4YöXöö﻿*ââéö)ö﻿ë&&£&öx>>$Z﻿ööââö﻿èô—â—>&


In [63]:
print(generate_text(model, start_string=u"Dracula", temp=1.0))

KeyboardInterrupt: 

In [ ]:
print(generate_text(model, start_string=u"song", temp=0.5))

In [ ]:
print(generate_text(model, start_string=u"song", temp=0.8))

In [ ]:
print(generate_text(model, start_string=u"song", temp=1.0))